# ACTUS STK and COM Contracts: Stock and Commodity

This notebook demonstrates the **Stock (STK)** and **Commodity (COM)** contract types from the ACTUS standard.

## What are STK and COM Contracts?

- **STK (Stock)**: Represents equity securities with potential dividends
- **COM (Commodity)**: Represents physical or financial commodities (gold, oil, wheat, etc.)

Both are simple contract types that track the value of an underlier.

In [ ]:
from jactus.contracts import create_contract
from jactus.core import (
    ContractAttributes,
    ContractType,
    ContractRole,
    ActusDateTime,
)
from jactus.observers import ConstantRiskFactorObserver

## Part 1: Stock (STK) Contract

### Example: Technology Stock Position

In [ ]:
# Contract dates
status_date = ActusDateTime(2024, 1, 1, 0, 0, 0)

# Create stock contract
stock_attrs = ContractAttributes(
    contract_id="STK-AAPL-001",
    contract_type=ContractType.STK,
    contract_role=ContractRole.RPA,  # Long position (own the stock)
    status_date=status_date,
    currency="USD",
    quantity=100.0,  # 100 shares
    # Stock-specific attributes can be added via market_object_code
    market_object_code="AAPL",  # Reference to stock ticker
)

# Create risk factor observer (provides stock price)
rf_observer = ConstantRiskFactorObserver(constant_value=150.0)  # Stock price $150

# Create stock contract
stock = create_contract(stock_attrs, rf_observer)

print("Stock Contract Created:")
print(f"  Symbol: AAPL")
print(f"  Shares: {stock_attrs.quantity}")
print(f"  Current Price: $150.00")
print(f"  Position Value: ${stock_attrs.quantity * 150.0:,.2f}")

### Initialize Stock State

In [ ]:
# Initialize state
stock_state = stock.initialize_state()

print("\nStock Initial State:")
print(f"  Status Date: {stock_state.sd.year}-{stock_state.sd.month:02d}-{stock_state.sd.day:02d}")
print(f"  Notional (quantity × price): ${float(stock_state.nt):,.2f}")
print(f"  Performance: {stock_state.prf.value}")

## Part 2: Commodity (COM) Contract

### Example: Gold Position

In [ ]:
# Create commodity contract
commodity_attrs = ContractAttributes(
    contract_id="COM-GOLD-001",
    contract_type=ContractType.COM,
    contract_role=ContractRole.RPA,  # Long position (own the commodity)
    status_date=status_date,
    currency="USD",
    quantity=10.0,  # 10 troy ounces
    market_object_code="XAU",  # Gold symbol
    unit="oz",  # Troy ounce
)

# Create risk factor observer (provides gold price per oz)
gold_rf_observer = ConstantRiskFactorObserver(constant_value=2000.0)  # $2000/oz

# Create commodity contract
gold = create_contract(commodity_attrs, gold_rf_observer)

print("\nCommodity Contract Created:")
print(f"  Commodity: Gold (XAU)")
print(f"  Quantity: {commodity_attrs.quantity} troy ounces")
print(f"  Current Price: $2,000.00/oz")
print(f"  Position Value: ${commodity_attrs.quantity * 2000.0:,.2f}")

### Initialize Commodity State

In [ ]:
# Initialize state
gold_state = gold.initialize_state()

print("\nGold Initial State:")
print(f"  Status Date: {gold_state.sd.year}-{gold_state.sd.month:02d}-{gold_state.sd.day:02d}")
print(f"  Notional (quantity × price): ${float(gold_state.nt):,.2f}")
print(f"  Performance: {gold_state.prf.value}")

## Part 3: Price Movement Simulation

Let's see how positions change with price movements

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Stock price scenarios
stock_prices = np.linspace(100, 200, 50)
stock_shares = 100
stock_values = stock_prices * stock_shares
stock_pnl = (stock_prices - 150) * stock_shares  # P&L vs initial $150

# Gold price scenarios  
gold_prices = np.linspace(1600, 2400, 50)
gold_oz = 10
gold_values = gold_prices * gold_oz
gold_pnl = (gold_prices - 2000) * gold_oz  # P&L vs initial $2000

# Create plots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# Stock position value
ax1.plot(stock_prices, stock_values, linewidth=2, color='blue')
ax1.axvline(x=150, color='red', linestyle='--', alpha=0.5, label='Initial Price')
ax1.set_xlabel('Stock Price ($)', fontsize=11)
ax1.set_ylabel('Position Value ($)', fontsize=11)
ax1.set_title('Stock Position Value (100 shares)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Stock P&L
ax2.fill_between(stock_prices, 0, stock_pnl, where=(stock_pnl >= 0), 
                 interpolate=True, alpha=0.3, color='green', label='Profit')
ax2.fill_between(stock_prices, 0, stock_pnl, where=(stock_pnl < 0), 
                 interpolate=True, alpha=0.3, color='red', label='Loss')
ax2.plot(stock_prices, stock_pnl, linewidth=2, color='black')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.axvline(x=150, color='red', linestyle='--', alpha=0.5)
ax2.set_xlabel('Stock Price ($)', fontsize=11)
ax2.set_ylabel('P&L ($)', fontsize=11)
ax2.set_title('Stock P&L (vs $150 entry)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gold position value
ax3.plot(gold_prices, gold_values, linewidth=2, color='gold')
ax3.axvline(x=2000, color='red', linestyle='--', alpha=0.5, label='Initial Price')
ax3.set_xlabel('Gold Price ($/oz)', fontsize=11)
ax3.set_ylabel('Position Value ($)', fontsize=11)
ax3.set_title('Gold Position Value (10 oz)', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gold P&L
ax4.fill_between(gold_prices, 0, gold_pnl, where=(gold_pnl >= 0), 
                 interpolate=True, alpha=0.3, color='green', label='Profit')
ax4.fill_between(gold_prices, 0, gold_pnl, where=(gold_pnl < 0), 
                 interpolate=True, alpha=0.3, color='red', label='Loss')
ax4.plot(gold_prices, gold_pnl, linewidth=2, color='black')
ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax4.axvline(x=2000, color='red', linestyle='--', alpha=0.5)
ax4.set_xlabel('Gold Price ($/oz)', fontsize=11)
ax4.set_ylabel('P&L ($)', fontsize=11)
ax4.set_title('Gold P&L (vs $2000 entry)', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Charts show:")
print("  • Top row: Stock position value and P&L")
print("  • Bottom row: Gold position value and P&L")
print("  • Linear relationship between price and position value")

## Part 4: Use as Underliers for Derivatives

STK and COM contracts are often used as underliers for derivative contracts

In [ ]:
print("Common Uses:")
print("\nSTK (Stock):")
print("  • Underlier for stock options (OPTNS)")
print("  • Underlier for equity futures (FUTUR)")
print("  • Component in equity portfolios")
print("  • Reference for equity derivatives")

print("\nCOM (Commodity):")
print("  • Underlier for commodity options")
print("  • Underlier for commodity futures")
print("  • Physical commodity positions")
print("  • Reference for commodity derivatives")

print("\nActual Example:")
print("  A call option on AAPL stock would reference the STK contract as its underlier")
print("  A gold futures contract would reference the COM-GOLD contract as its underlier")

## Key Observations

STK and COM contracts implement:

1. **Simple Position Tracking**:
   - Value = Quantity × Price
   - No complex cash flows
   - Direct price exposure

2. **Role in ACTUS Ecosystem**:
   - Serve as underliers for derivative contracts
   - Provide price references
   - Enable composite contract structures

3. **Differences**:
   - STK: May include dividends (via dividend payment cycles)
   - COM: Pure commodity exposure, no cash flows

4. **ACTUS Compliance**: Follow ACTUS v1.1 specification

## References

- ACTUS Specification v1.1, Section 7.9 - Stock (STK)
- ACTUS Specification v1.1, Section 7.10 - Commodity (COM)
- [ACTUS Technical Specification](https://www.actusfrf.org/)